In [100]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna

In [101]:
#creates the training dataset
df_train = pd.read_csv("data/final/final_training.csv")
df_train = df_train[df_train['pot_doc'] != '[]']
X_train_df=pd.DataFrame(data = np.array(df), columns = df.columns).drop(columns="documentid").drop(columns="Unnamed: 0")
y_train_df = pd.DataFrame(data = np.array(df['documentid']), columns = ['documentid'])

#creates the testing dataset
df_train = pd.read_csv("data/final/final_test.csv")
X_test_df=pd.DataFrame(data = np.array(df), columns = df.columns).drop(columns="documentid").drop(columns="Unnamed: 0")
y_test_df = pd.DataFrame(data = np.array(df['documentid']), columns = ['documentid'])


In [102]:
def compute_accuracy(x, y, z,a):
    results = []
    for row in X_train_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*(100-eval(row[1]['pot_price'])[idx])/100)**2 
            + (y*(1-eval(row[1]['pot_date'])[idx]))**2)
            + (z*(100-eval(row[1]['pot_address'])[idx])/100)**2
            #+ (a*eval(row[1]['pot_name'])[idx])**2
            
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return np.sum(results == np.array(y_train_df['documentid']))/len(X_train_df)
    
def compute_accuracy2(x, y, z, a):
    results = []
    for row in X_test_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*(100-eval(row[1]['pot_price'])[idx])/100)**2 
            + (y*(1-eval(row[1]['pot_date'])[idx]))**2
            + (z*(100-eval(row[1]['pot_address'])[idx])/100)**2
            #+ (a*eval(row[1]['pot_name'])[idx])**2
            )
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return results

In [103]:

def objective(trial):
    x = trial.suggest_float('x', 0, 10)
    y = trial.suggest_float('y', 0, 10)
    z = trial.suggest_float('z', 0, 10)
    #a = trial.suggest_float('a', 0, 10)
    return 1-compute_accuracy(x,y,z,0)

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params  # E.g. {'x': 2.002108042}



[I 2023-01-29 07:40:32,362] A new study created in memory with name: no-name-e0a767c4-af09-4c02-ac5b-2e1913ae27eb
[I 2023-01-29 07:40:32,612] Trial 0 finished with value: 0.16801619433198378 and parameters: {'x': 6.570318854726422, 'y': 7.781333003394635, 'z': 2.000563672773532}. Best is trial 0 with value: 0.16801619433198378.
[I 2023-01-29 07:40:32,844] Trial 1 finished with value: 0.16801619433198378 and parameters: {'x': 1.527916338738975, 'y': 0.42560099262255013, 'z': 6.403317704718856}. Best is trial 0 with value: 0.16801619433198378.
[I 2023-01-29 07:40:33,098] Trial 2 finished with value: 0.16801619433198378 and parameters: {'x': 4.375325172990934, 'y': 7.689626739724722, 'z': 0.4035831063603301}. Best is trial 0 with value: 0.16801619433198378.
[I 2023-01-29 07:40:33,334] Trial 3 finished with value: 0.16801619433198378 and parameters: {'x': 5.734738042264006, 'y': 6.442943356214837, 'z': 6.976713609007506}. Best is trial 0 with value: 0.16801619433198378.
[I 2023-01-29 07:40

{'x': 6.570318854726422, 'y': 7.781333003394635, 'z': 2.000563672773532}

In [106]:
res = compute_accuracy2(study.best_params['x'], 1, study.best_params['y'], 0)

real = list(y_test_df['documentid'])


In [105]:
'''
calculates the percentage where our predicted data conforms to our acutal data
'''
def calculate_percentage(list_a, list_b):
    common_elements = set(list_a) & set(list_b)
    percentage = (len(common_elements) / len(list_a)) * 100
    return percentage
    
#from our model, we have a predicted accuracy of 84.21052631578947
print(calculate_percentage(res,real))

84.21052631578947
